# Chapter 2: Working with Chains

Goals:
- Understand LLMChain (prompt → LLM → output)
- Use multiple variables in prompts
- Build simple sequential two-step flows
- See how to name outputs (SequentialChain)
- Use `verbose=True` to inspect prompts & intermediate outputs

Assumes: you already completed `01_intro` and have your `.env` with `GOOGLE_API_KEY`.


In [1]:
# import the api keys
import os
from dotenv import load_dotenv

load_dotenv()

google_key = os.environ.get("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain , SimpleSequentialChain, SequentialChain
from langchain_core.prompts import PromptTemplate

# initialize the LLM

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    api_key = google_key,
    temperature = 0.5
)

print(llm)

model='models/gemini-1.5-flash' google_api_key=SecretStr('**********') temperature=0.5 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001CDDE939190> default_metadata=() model_kwargs={}


## 2.1 Recap: Single LLMChain

In [3]:
prompt = PromptTemplate.from_template(
    """
    Suggest a name for a {sport} club based in {country}.
    """
)

chain1 = LLMChain(
    llm = llm,
    prompt = prompt
)

output = chain1.invoke({
    "sport":"football",
    "country":"England"
})

print(output["text"])

C:\Users\amalv\AppData\Local\Temp\ipykernel_1052\1882250279.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(


Here are a few suggestions, playing with different styles:

**Traditional & Geographic:**

* **Ashford Town FC:** (If based near Ashford)  Simple, classic.
* **The Severnside Rovers:** (If near the River Severn) Evokes a sense of place and movement.
* **North Downs United:** (If near the North Downs)  Geographic and suggests unity.
* **Wessex Wanderers:** (If in the Wessex region)  Historical and adventurous.


**Modern & Catchy:**

* **City Phoenix FC:** Modern, suggests rebirth or rising from the ashes.
* **Steel City United:** (If in a steel-producing area)  Strong, industrial feel.
* **The Albion Eagles:**  Combines a classic feel with a powerful image.
* **Vanguard FC:**  Modern, suggests forward-thinking and ambition.


**Unique & Quirky:**

* **The Clockwork Crows:**  Unusual and memorable.
* **The Crimson Tide FC:**  Evocative and memorable.
* **The Ironclad Rovers:**  Strong and adventurous.


Remember to check if the name is already in use by another club before settling on o

## 2.2 SimpleSequenceChain
- Used when chain 1 produces a single output which can be used as input for next chain

In [6]:
prompt_a = PromptTemplate.from_template("Describe a {concept} in 1 sentence.")
chain1 = LLMChain(
    llm = llm,
    prompt = prompt_a
)
# chain B uses output from chain1
prompt_b = PromptTemplate.from_template("Suggest one practical use for this: \n\n{input}")
chain2 = LLMChain(
    llm = llm,
    prompt = prompt_b
)

#Turn it into a simple sequential chain
seq = SimpleSequentialChain(
    chains = [chain1 , chain2],
    verbose = True
)

print(seq.invoke("Total Internal Reflection"))



> Entering new SimpleSequentialChain chain...
Total internal reflection occurs when light traveling from a denser medium to a less dense medium strikes the interface at an angle greater than the critical angle, causing all the light to be reflected back into the denser medium.
Fiber optic cables.  Total internal reflection is the fundamental principle allowing light signals to travel long distances down a fiber optic cable with minimal loss.  The light is continuously reflected internally within the core of the fiber, preventing it from escaping.

> Finished chain.
{'input': 'Total Internal Reflection', 'output': 'Fiber optic cables.  Total internal reflection is the fundamental principle allowing light signals to travel long distances down a fiber optic cable with minimal loss.  The light is continuously reflected internally within the core of the fiber, preventing it from escaping.'}


## 2.3 SequentialChain with named variables

* output_key on each LLMChain assigns the chain's output into a named key in the overall state.
* SequentialChain lets you explicitly define which top-level inputs are required and which outputs you want back.
* The result is typically a dict with keys listed in output_variables

In [9]:
#chain 1
prompt1 = PromptTemplate.from_template("Desribe this language {language} in one sentance. You must not use the word {language} in the result")
chain1 = LLMChain(
    llm = llm,
    prompt = prompt1,
    output_key = "concept_info"
    )


#chain 2
prompt2 =PromptTemplate.from_template("What is the technology used in this \n {concept_info}.")
chain2 = LLMChain(
    llm = llm,
    prompt = prompt2,
    output_key = "result"
)

# Sequential chain

seq_chain = SequentialChain(
    chains = [chain1 , chain2],
    input_variables = ["language"],
    output_variables = ["concept_info" , "result"],
    verbose = True
)

result = seq_chain.invoke({"language": "java"})
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'language': 'java', 'concept_info': "It's a general-purpose, class-based, object-oriented programming language that's widely used for developing applications on various platforms.", 'result': "That description points to **many object-oriented programming languages**, not a single specific technology.  Many languages fit that description, including (but not limited to):\n\n* **Java:**  A very popular choice fitting the description perfectly.\n* **C#:**  Another widely used language, especially in Microsoft environments.\n* **Python:** While often used in a more dynamic style, Python also supports object-oriented programming and is general-purpose.\n* **C++:** A powerful language that's class-based and object-oriented, though it also allows procedural programming.\n* **PHP:** Though often associated with web development, PHP is also a general-purpose, class-based, object-oriented language.\n\n\nTo know the *specific* technolog

In [11]:
print(result["result"])

That description points to **many object-oriented programming languages**, not a single specific technology.  Many languages fit that description, including (but not limited to):

* **Java:**  A very popular choice fitting the description perfectly.
* **C#:**  Another widely used language, especially in Microsoft environments.
* **Python:** While often used in a more dynamic style, Python also supports object-oriented programming and is general-purpose.
* **C++:** A powerful language that's class-based and object-oriented, though it also allows procedural programming.
* **PHP:** Though often associated with web development, PHP is also a general-purpose, class-based, object-oriented language.


To know the *specific* technology, you'd need more information about the application or system it's used in.


## 2.4 Inspecting the built prompt
format() renders the prompt template locally — useful for checking variable names & formatting before a remote API call.

In [12]:
# If you want to see the final prompt BEFORE calling the LLM:
pt = PromptTemplate.from_template("Describe {thing} in one line.")
print("Rendered prompt text:")
print(pt.format(thing="an island made of glass"))   # show the filled string

Rendered prompt text:
Describe an island made of glass in one line.


## 2.4 Practice tasks

A) Multi-variable prompt:
   - Prompt: "Write a menu item name and a 1-line description for a {cuisine} dish with {main_ingredient}."
   - Use LLMChain and test with 3 cuisines.

B) Two-step pipeline:
   - Chain 1: Given {genre}, write a short plot (2–3 sentences).
   - Chain 2: Given the plot, produce 3 taglines.
   - Use SequentialChain, set output_keys, run with verbose=True.


## A

In [14]:
prompt = PromptTemplate.from_template(
    "Write a menu item name and a 1-line description for a {cusine} dish with {main_ingredient}"
)

chain = LLMChain(
    llm = llm,
    prompt = prompt
)

print(chain.invoke({"cusine":"Indian" , "main_ingredient": "sea food"})["text"])

**Name:**  Coastal Curry Catch

**Description:**  Succulent seafood simmered in a vibrant coconut-tomato curry with fragrant spices.


## B

In [19]:
prompt1 = PromptTemplate.from_template("Given {genre}, write a short plot (2-3 sentences).")
chain1 = LLMChain(
    llm = llm,
    prompt = prompt1,
    output_key="plot"
)
prompt2 = PromptTemplate.from_template("Produce three taglines using this plot:\n{plot}")
chain2 = LLMChain(
    llm = llm,
    prompt = prompt2,
    output_key = "result"
)

seq = SequentialChain(
    chains = [chain1, chain2],
    input_variables = ["genre"],
    output_variables = ["plot" , "result"],
    verbose = True
)

output = seq.invoke({"genre":"horror"})
print(output["result"])



> Entering new SequentialChain chain...

> Finished chain.
1. **The fog hides more than just the island. It hides the truth.** (Focuses on mystery and suspense)
2. **Their smiles are perfect. Their secret is terrifying.** (Highlights the deceptive nature of the creatures and the horror)
3. **Memory is his only weapon.  Survival is his only hope.** (Emphasizes the protagonist's struggle and the high stakes)
